# Step 1: Importing Essential Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Step 2: Loading data and Making labels

In [2]:
#Dataset used: https://www.kaggle.com/kmader/skin-cancer-mnist-ham10000
path='archive/hmnist_28_28_RGB.csv'
meta_path = "archive/HAM10000_metadata.csv"

In [3]:
df = pd.read_csv(path)
metadata = pd.read_csv(meta_path)

In [4]:
#drop NA values
meta_set = metadata[["age", "sex", "localization"]]
df = pd.concat([meta_set, df], axis=1)
df = df.dropna()
df = df.iloc[:,3:]
df.tail()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
10010,183,165,181,182,165,180,184,166,182,188,...,208,185,187,208,186,186,206,187,189,0
10011,2,3,1,38,33,32,121,104,103,132,...,96,79,76,24,23,21,3,4,1,0
10012,132,118,118,167,149,149,175,156,160,184,...,204,181,178,181,159,153,172,151,145,0
10013,160,124,146,164,131,152,167,127,146,169,...,185,162,167,184,157,166,185,162,172,0
10014,175,142,121,181,150,134,181,150,133,178,...,159,79,82,174,137,125,175,139,126,6


# Step 3: Train Test Split

In [5]:
# Split into train, validation, and test.
np.random.seed(2070404)

# Shuffle all records.
df_shuffle = df.sample(frac = 1)

# Create split counts.
splits = np.multiply(len(df_shuffle), (0.6,0.2,0.2)).astype(int)
print(f"Split counts (train/ validation/ test): {splits}")

# Create split data sets.
train_set, valid_set, test_set = np.split(df_shuffle, [splits[0], splits[0] + splits[1]])

# Reset split set indicies.
train_set.reset_index(drop = True, inplace = True)
valid_set.reset_index(drop = True, inplace = True)
test_set.reset_index(drop = True, inplace = True)

Split counts (train/ validation/ test): [5974 1991 1991]


In [6]:
print(len(train_set))

5974


In [7]:
print(len(test_set))

1993


In [8]:
df.label.unique()

array([2, 4, 3, 6, 5, 1, 0])

In [10]:
#separate features and labels
y_train = train_set['label']
x_train = train_set.drop(columns=['label'])
y_valid = valid_set['label']
x_valid = valid_set.drop(columns=['label'])
y_test = test_set['label']
x_test = test_set.drop(columns=['label'])

columns = list(x_train)

# Step 4: Exploratory Data Analysis and Preprocessing

In [11]:
import matplotlib.pyplot as plt
import random

#reshape images from 1D to RGB
num=random.randint(0,6000)
x_train = np.array(x_train, dtype=np.uint8).reshape(-1,28,28,3)
x_valid = np.array(x_valid, dtype=np.uint8).reshape(-1,28,28,3)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow import keras
import tensorflow as tf

2023-12-05 22:20:08.567443: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Step 5: Model Building (CNN)

In [13]:
def build_model(kernel_size = 2, pool_size = 2, learning_rate = 0.001, optimizer_name = "Adam", additional_dense = True):
    """
    Build a CNN model using Keras.

    Args:
    kernel_size: convolution layer kernel size
    pool_size: pooling layer pool size
    learning_rate: optimizer learning rate
    optimizer_name: optimizer used to compile the model
    additional_dense: whether to add an additional dense layer

    Returns:
    model: A tf.keras model
    """
    
    #clear session
    tf.keras.backend.clear_session()
    model = tf.keras.Sequential()

    #add input layer
    model.add(keras.layers.Input(shape=[28, 28, 3]))

    #add convolution layer 1
    model.add(tf.keras.layers.Conv2D(
        filters = 32, kernel_size = (kernel_size, kernel_size),
        strides=(1,1), padding='same',
        data_format = 'channels_last',
        input_shape = (28, 28, 3),  # Updated input shape for RGB
        name='conv_1', activation='relu'))

    #add pooling layer 1
    model.add(tf.keras.layers.MaxPool2D(
        pool_size= (pool_size, pool_size), name = 'pool_1'))

    #add convolution layer 2
    model.add(tf.keras.layers.Conv2D(
        filters = 64, kernel_size = (kernel_size, kernel_size),
        strides = (1,1), padding = 'same',
        name = 'conv_2', activation = 'relu'))

    #add pooling layer 2
    model.add(tf.keras.layers.MaxPool2D(
        pool_size = (pool_size, pool_size), name = 'pool_2'))

    #add flattening layer
    model.add(tf.keras.layers.Flatten())


    #add dense layer 1
    model.add(tf.keras.layers.Dense(
        units = 1024, name = 'fc_1', 
        activation = 'relu'))

    #dropout
    model.add(tf.keras.layers.Dropout(rate = 0.5))
    
    if additional_dense:
        #add dense layer 2
        model.add(tf.keras.layers.Dense(
            units = 1024, name = 'fc_2', 
            activation = 'relu'))

    #add output layer
    model.add(tf.keras.layers.Dense(
        units = 7, name = 'fc_3',
        activation = 'softmax'))

    if optimizer_name == "Adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
    elif optimizer_name == "SGD":
        optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate)
    elif optimizer_name == "Adagrad":
        optimizer = tf.keras.optimizers.Adagrad(learning_rate = learning_rate)

    #compile model
    model.compile(loss = 'sparse_categorical_crossentropy',
                  optimizer = optimizer,
                  metrics = ['accuracy'])
                
    return model


# Step 6: Fitting the Model with Different Hyper-parameters

### 6.1: fit models with and without an additional dense layer with different kernel sizes

In [14]:
from datetime import datetime

#list of kernel sizes to fit
kernel_list = [1, 2, 3, 4, 5]

kernel_dict = {}

for i in kernel_list:
    
    #fit model with an additional dense layer
    start_time = datetime.now()
    print("-----kernel size: " + str(i) + "-----")
    model = build_model(kernel_size = i, pool_size = 2, learning_rate = 0.001, optimizer_name = "Adam", additional_dense = True)
    history_true = model.fit(x_train,
                        y_train,
                        validation_data=(x_valid, y_valid),
                        batch_size = 64,
                        epochs = 10)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    #fit model without an additional dense layer
    start_time = datetime.now()
    print("-----kernel size: " + str(i) + "-----")
    model = build_model(kernel_size = i, pool_size = 2, learning_rate = 0.001, optimizer_name = "Adam", additional_dense = False)
    history_false = model.fit(x_train,
                        y_train,
                        validation_data=(x_valid, y_valid),
                        batch_size = 64,
                        epochs = 10)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    #store training results
    kernel_dict[i] = [history_true.history['accuracy'][-1], history_true.history['val_accuracy'][-1], history_false.history['accuracy'][-1], history_false.history['val_accuracy'][-1]]
    

-----kernel size: 1-----
Epoch 1/10
94/94 [==============================] - 4s 37ms/step - loss: 14.9724 - accuracy: 0.5820 - val_loss: 1.0136 - val_accuracy: 0.6735
Epoch 2/10
94/94 [==============================] - 3s 36ms/step - loss: 1.0515 - accuracy: 0.6682 - val_loss: 1.0082 - val_accuracy: 0.6735
Epoch 3/10
94/94 [==============================] - 3s 36ms/step - loss: 1.0456 - accuracy: 0.6661 - val_loss: 0.9928 - val_accuracy: 0.6735
Epoch 4/10
94/94 [==============================] - 3s 36ms/step - loss: 1.0349 - accuracy: 0.6682 - val_loss: 0.9908 - val_accuracy: 0.6735
Epoch 5/10
94/94 [==============================] - 3s 36ms/step - loss: 1.0179 - accuracy: 0.6687 - val_loss: 0.9801 - val_accuracy: 0.6735
Epoch 6/10
94/94 [==============================] - 3s 36ms/step - loss: 1.0268 - accuracy: 0.6687 - val_loss: 0.9624 - val_accuracy: 0.6735
Epoch 7/10
94/94 [==============================] - 3s 36ms/step - loss: 1.0143 - accuracy: 0.6676 - val_loss: 0.9717 - val_accu

### 6.2: fit models with and without an additional dense layer with different pool sizes

In [15]:
#list of pool sizes to fit
pool_list = [1, 2, 3, 4, 5]

pool_dict = {}

for i in pool_list:

    #fit model with an additional dense layer
    start_time = datetime.now()
    print("-----pool size: " + str(i) + "-----")
    model = build_model(kernel_size = 2, pool_size = i, learning_rate = 0.001, optimizer_name = "Adam", additional_dense = True)
    history_true = model.fit(x_train,
                        y_train,
                        validation_data=(x_valid, y_valid),
                        batch_size = 64,
                        epochs = 10)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    #fit model without an additional dense layer
    start_time = datetime.now()
    print("-----pool size: " + str(i) + "-----")
    model = build_model(kernel_size = 2, pool_size = i, learning_rate = 0.001, optimizer_name = "Adam", additional_dense = False)
    history_false = model.fit(x_train,
                        y_train,
                        validation_data=(x_valid, y_valid),
                        batch_size = 64,
                        epochs = 10)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    #store results
    pool_dict[i] = [history_true.history['accuracy'][-1], history_true.history['val_accuracy'][-1], history_false.history['accuracy'][-1], history_false.history['val_accuracy'][-1]]


-----pool size: 1-----
Epoch 1/10
94/94 [==============================] - 43s 448ms/step - loss: 97.8536 - accuracy: 0.6038 - val_loss: 0.9714 - val_accuracy: 0.6640
Epoch 2/10
94/94 [==============================] - 42s 452ms/step - loss: 1.0031 - accuracy: 0.6709 - val_loss: 0.9627 - val_accuracy: 0.6735
Epoch 3/10
94/94 [==============================] - 42s 445ms/step - loss: 0.9435 - accuracy: 0.6758 - val_loss: 0.9353 - val_accuracy: 0.6710
Epoch 4/10
94/94 [==============================] - 42s 447ms/step - loss: 0.8788 - accuracy: 0.6890 - val_loss: 0.8902 - val_accuracy: 0.6775
Epoch 5/10
94/94 [==============================] - 42s 445ms/step - loss: 0.8476 - accuracy: 0.6977 - val_loss: 0.8551 - val_accuracy: 0.6956
Epoch 6/10
94/94 [==============================] - 42s 446ms/step - loss: 0.8094 - accuracy: 0.7166 - val_loss: 0.8618 - val_accuracy: 0.7022
Epoch 7/10
94/94 [==============================] - 42s 444ms/step - loss: 0.7589 - accuracy: 0.7189 - val_loss: 0.873

### 6.3: fit models with and without an additional dense layer with different learning rates

In [16]:
#list of learning rates to fit
learning_list = [0.001, 0.01, 0.1, 1]

learning_dict = {}

for i in learning_list:

    #fit model with an additional dense layer
    start_time = datetime.now()
    print("-----learning_rate: " + str(i) + "-----")
    model = build_model(kernel_size = 2, pool_size = 2, learning_rate = i, optimizer_name = "Adam", additional_dense = True)
    history_true = model.fit(x_train,
                        y_train,
                        validation_data=(x_valid, y_valid),
                        batch_size = 64,
                        epochs = 10)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    #fit model without an additional dense layer
    start_time = datetime.now()
    print("-----learning_rate: " + str(i) + "-----")
    model = build_model(kernel_size = 2, pool_size = 2, learning_rate = i, optimizer_name = "Adam", additional_dense = False)
    history_false = model.fit(x_train,
                        y_train,
                        validation_data=(x_valid, y_valid),
                        batch_size = 64,
                        epochs = 10)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    #store results
    learning_dict[i] = [history_true.history['accuracy'][-1], history_true.history['val_accuracy'][-1], history_false.history['accuracy'][-1], history_false.history['val_accuracy'][-1]]
    

-----learning_rate: 0.001-----
Epoch 1/10
94/94 [==============================] - 4s 41ms/step - loss: 9.9540 - accuracy: 0.6138 - val_loss: 0.9272 - val_accuracy: 0.6755
Epoch 2/10
94/94 [==============================] - 4s 40ms/step - loss: 0.9500 - accuracy: 0.6691 - val_loss: 0.9033 - val_accuracy: 0.6791
Epoch 3/10
94/94 [==============================] - 4s 40ms/step - loss: 0.9241 - accuracy: 0.6701 - val_loss: 0.8499 - val_accuracy: 0.6956
Epoch 4/10
94/94 [==============================] - 4s 40ms/step - loss: 0.8891 - accuracy: 0.6754 - val_loss: 0.8491 - val_accuracy: 0.6971
Epoch 5/10
94/94 [==============================] - 4s 40ms/step - loss: 0.8755 - accuracy: 0.6843 - val_loss: 0.9018 - val_accuracy: 0.6821
Epoch 6/10
94/94 [==============================] - 4s 40ms/step - loss: 0.8603 - accuracy: 0.6878 - val_loss: 0.8441 - val_accuracy: 0.6996
Epoch 7/10
94/94 [==============================] - 4s 40ms/step - loss: 0.8372 - accuracy: 0.6989 - val_loss: 0.8841 - val

### 6.4: fit models with and without an additional dense layer with different batch sizes

In [17]:
#list of batch sizes to fit
batch_list = [32, 64, 128, 256]

batch_dict = {}

for i in batch_list:

    #fit model with an additional dense layer
    start_time = datetime.now()
    print("-----batch size: " + str(i) + "-----")
    model = build_model(kernel_size = 2, pool_size = 2, learning_rate = 0.001, optimizer_name = "Adam", additional_dense = True)
    history_true = model.fit(x_train,
                        y_train,
                        validation_data=(x_valid, y_valid),
                        batch_size = i,
                        epochs = 10)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    #fit model without an additional dense layer
    start_time = datetime.now()
    print("-----batch size: " + str(i) + "-----")
    model = build_model(kernel_size = 2, pool_size = 2, learning_rate = 0.001, optimizer_name = "Adam", additional_dense = False)
    history_false = model.fit(x_train,
                        y_train,
                        validation_data=(x_valid, y_valid),
                        batch_size = i,
                        epochs = 10)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    #store results
    batch_dict[i] = [history_true.history['accuracy'][-1], history_true.history['val_accuracy'][-1], history_false.history['accuracy'][-1], history_false.history['val_accuracy'][-1]]
    

-----batch size: 32-----
Epoch 1/10
187/187 [==============================] - 7s 34ms/step - loss: 7.3976 - accuracy: 0.6348 - val_loss: 0.9447 - val_accuracy: 0.6730
Epoch 2/10
187/187 [==============================] - 6s 33ms/step - loss: 0.9815 - accuracy: 0.6692 - val_loss: 0.9113 - val_accuracy: 0.6665
Epoch 3/10
187/187 [==============================] - 6s 34ms/step - loss: 0.9348 - accuracy: 0.6722 - val_loss: 0.8702 - val_accuracy: 0.6971
Epoch 4/10
187/187 [==============================] - 6s 34ms/step - loss: 0.8800 - accuracy: 0.6855 - val_loss: 0.8346 - val_accuracy: 0.6991
Epoch 5/10
187/187 [==============================] - 6s 33ms/step - loss: 0.8489 - accuracy: 0.6940 - val_loss: 0.8884 - val_accuracy: 0.6720
Epoch 6/10
187/187 [==============================] - 6s 34ms/step - loss: 0.8049 - accuracy: 0.7041 - val_loss: 0.8017 - val_accuracy: 0.7052
Epoch 7/10
187/187 [==============================] - 6s 33ms/step - loss: 0.8343 - accuracy: 0.6999 - val_loss: 0.82

### 6.5: fit models with and without an additional dense layer with different optimizers

In [18]:
#list of optimziers to fit
optimizer_list = ["Adam", "SGD", "Adagrad"]

optimizer_dict = {}

for i in optimizer_list:

    #fit model with an additional layer
    start_time = datetime.now()
    print("-----optimizer: " + i + "-----")
    model = build_model(kernel_size = 2, pool_size = 2, learning_rate = 0.001, optimizer_name = i, additional_dense = True)
    history_true = model.fit(x_train,
                        y_train,
                        validation_data=(x_valid, y_valid),
                        batch_size = 64,
                        epochs = 10)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    #fit model without an additional layer
    start_time = datetime.now()
    print("-----optimizer: " + i + "-----")
    model = build_model(kernel_size = 2, pool_size = 2, learning_rate = 0.001, optimizer_name = i, additional_dense = False)
    history_false = model.fit(x_train,
                        y_train,
                        validation_data=(x_valid, y_valid),
                        batch_size = 64,
                        epochs = 10)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    optimizer_dict[i] = [history_true.history['accuracy'][-1], history_true.history['val_accuracy'][-1], history_false.history['accuracy'][-1], history_false.history['val_accuracy'][-1]]

    

-----optimizer: Adam-----
Epoch 1/10
94/94 [==============================] - 4s 40ms/step - loss: 17.1058 - accuracy: 0.6016 - val_loss: 0.9459 - val_accuracy: 0.6665
Epoch 2/10
94/94 [==============================] - 4s 42ms/step - loss: 0.9777 - accuracy: 0.6729 - val_loss: 0.9301 - val_accuracy: 0.6856
Epoch 3/10
94/94 [==============================] - 4s 40ms/step - loss: 0.9244 - accuracy: 0.6803 - val_loss: 0.9259 - val_accuracy: 0.6931
Epoch 4/10
94/94 [==============================] - 4s 40ms/step - loss: 0.9271 - accuracy: 0.6736 - val_loss: 0.9036 - val_accuracy: 0.6901
Epoch 5/10
94/94 [==============================] - 4s 40ms/step - loss: 0.8723 - accuracy: 0.6900 - val_loss: 0.8453 - val_accuracy: 0.7047
Epoch 6/10
94/94 [==============================] - 4s 40ms/step - loss: 0.8784 - accuracy: 0.6915 - val_loss: 0.9057 - val_accuracy: 0.6479
Epoch 7/10
94/94 [==============================] - 4s 39ms/step - loss: 0.8518 - accuracy: 0.6974 - val_loss: 0.8110 - val_acc

# 7: output results

In [19]:
import csv

kernel_table = []

for key in kernel_dict:
    kernel_table.append([key, kernel_dict[key][0], kernel_dict[key][1], kernel_dict[key][2], kernel_dict[key][3]])

with open('clean_kernel.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(kernel_table)

In [20]:
pool_table = []

for key in pool_dict:
    pool_table.append([key, pool_dict[key][0], pool_dict[key][1], pool_dict[key][2], pool_dict[key][3]])

with open('clean_pool.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(pool_table)

In [21]:
learning_table = []

for key in learning_dict:
    learning_table.append([key, learning_dict[key][0], learning_dict[key][1], learning_dict[key][2], learning_dict[key][3]])

with open('clean_learning.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(learning_table)

In [22]:
batch_table = []

for key in batch_dict:
    batch_table.append([key, batch_dict[key][0], batch_dict[key][1], batch_dict[key][2], batch_dict[key][3]])

with open('clean_batch.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(batch_table)

In [23]:
optimizer_table = []

for key in optimizer_dict:
    optimizer_table.append([key, optimizer_dict[key][0], optimizer_dict[key][1], optimizer_dict[key][2], optimizer_dict[key][3]])

with open('clean_optimizer.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(optimizer_table)